In [3]:
import tensorflow.keras as keras
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from os import listdir
from numpy import reshape 
from numpy import array
from sklearn import preprocessing

#Define path for training sets
PATHOLOGICAL_ID = "Pathological-Training/"
ACCEPTABLE_ID = "Non-Pathological-Training"

#Define functions to read files contained in training set
def pathological_path(filename):
    return os.path.join(PATHOLOGICAL_ID, filename)

def acceptable_path(filename):
    return os.path.join(ACCEPTABLE_ID, filename)

#Name of the folder for training sets
Names_Pathological = os.listdir('Pathological-Training')
Names_Acceptable = os.listdir('Non-Pathological-Training')

#Open first file and define pandas data frame for the pathological portion of the training set 
infile = open(pathological_path(Names_Pathological[0]),'r')
press_pathological = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))

infile = open(acceptable_path(Names_Acceptable[0]),'r')
press_acceptable = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))


#Import training files and define Pandas data frame

for i in range(1,len(Names_Pathological)-1):
    infile = open(pathological_path(Names_Pathological[i]),'r')
    Press = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))
    press_pathological =press_pathological.append(pd.DataFrame(Press))
    

for i in range(1,len(Names_Acceptable)-1):
    infile = open(acceptable_path(Names_Acceptable[i]),'r')
    Press = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))
    press_acceptable =press_acceptable.append(pd.DataFrame(Press))


In [4]:
#Now we need to reshape the data to matrices

#Define grid
Tgrid = 771
muBgrid = 451
elements = Tgrid*muBgrid
    
#Produce matrices, the classification list contains 0 (pathological) or 1 (acceptable)
#corresponding to the matrix

for i in range(0,int(len(press_acceptable)/347721)):
    data = press_acceptable.iloc[i*elements : (i+1)*elements]['P']
    data = array(data)
#     data = data.reshape((data.shape[0], 1))
    data = data.reshape(Tgrid,muBgrid)
    if i ==0 :
        matpress = [data]
        classification = [1]
    else:
        matpress.append(data)
        classification.append(1)
    
for i in range(0,int(len(press_pathological)/347721)):
    data = press_pathological.iloc[i*elements : (i+1)*elements]['P']
    data = array(data)
#     data = data.reshape((data.shape[0], 1))
    data = data.reshape(Tgrid,muBgrid)
    matpress.append(data)
    classification.append(0)
    
### Now we need compute the SVD and extract the coefficients in the muB basis 

#first we want to take the transpose, then we want to center our matrices across the rows, then we compute the SVD

for i in range(0, len(matpress)):
    if i == 0:
        scalled_matpress = [preprocessing.scale(matpress[0].T).T] 
    else:
        scalled_matpress.append(preprocessing.scale(matpress[i].T).T)
        

In [5]:
# Compute SVD and first coefficient 
from matplotlib import pyplot as plt

x_train = []

for j in range(0,len(scalled_matpress)):
    scalled_matpress[j] = pd.DataFrame(scalled_matpress[j])
    scalled_matpress[j].dropna(inplace= True)
    u, s, vh = np.linalg.svd(scalled_matpress[j], full_matrices=True)
    scalled_matpress[j] = array(scalled_matpress[j])
    for i in range(0, scalled_matpress[j].shape[0]-1):
        if i == 0:
            values = [np.dot(vh[:][1],scalled_matpress[j][i][:])/np.dot(vh[:][1],vh[:][1])]
        else:
            values.append(np.dot(vh[:][1],scalled_matpress[j][i][:])/np.dot(vh[:][1],vh[:][1]))
    values = array(values)
    x_train.append(values)

    
#Correction to training data missing element

x_train[45] = np.append(x_train[45], x_train[45][:][768])

In [6]:
#Now we normalize the training data for the neural network

x_train = tf.keras.utils.normalize(x_train,axis=0)

classification = array(classification)

#Our data is ready to be fed to the neural net



In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(770, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(770, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, classification, epochs = 3,verbose = 2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
